In [3]:
from datetime import datetime

import spotipy
from spotipy.oauth2 import SpotifyOAuth

from config import *

AUTH_URL = "https://accounts.spotify.com/api/token"
API_ENDPOINT = "https://api.spotify.com/v1/"

scope = "user-library-read user-read-recently-played"

sp = spotipy.Spotify(
  auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=scope,
  )
)

In [4]:
# convert timestamp to datetime
datetime.fromtimestamp(1638567430791/1000)

datetime.datetime(2021, 12, 3, 22, 37, 10, 791000)

In [5]:
history: dict[str, dict] = sp.current_user_recently_played()

In [6]:
history['items'][0]

{'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3JhNCzhSMTxs9WLGJJxWOY'},
     'href': 'https://api.spotify.com/v1/artists/3JhNCzhSMTxs9WLGJJxWOY',
     'id': '3JhNCzhSMTxs9WLGJJxWOY',
     'name': 'Macklemore',
     'type': 'artist',
     'uri': 'spotify:artist:3JhNCzhSMTxs9WLGJJxWOY'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4myTppRgh0rojLxx8RycOp'},
     'href': 'https://api.spotify.com/v1/artists/4myTppRgh0rojLxx8RycOp',
     'id': '4myTppRgh0rojLxx8RycOp',
     'name': 'Ryan Lewis',
     'type': 'artist',
     'uri': 'spotify:artist:4myTppRgh0rojLxx8RycOp'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/5BcAKTbp20cv7tC5VqPFoC'},
     'href': 'https://api.spotify.com/v1/artists/5BcAKTbp20cv7tC5VqPFoC',
     'id': '5BcAKTbp20cv7tC5VqPFoC',
     'name': 'Macklemore & Ryan Lewis',
     'type': 'artist',
     'uri': 'spotify:artist:5BcAKTbp20cv7tC5VqPFoC'}],
   'ava

In [9]:
from collections import defaultdict

from tqdm import tqdm

total = sp.current_user_saved_tracks(limit=1)["total"]
offset = 0
artists_ids_count = defaultdict(int)
tracks = set()
with tqdm(total=266) as pbar:
  while offset < total:
    res = sp.current_user_saved_tracks(limit=50, offset=offset)["items"]
    # get ids of all artists in saved tracks
    for item in res:
      if item["added_at"] < "2021-01-01":
        break
      tracks.add(item["track"]["id"])
      for artist in item["track"]["artists"]:
        artists_ids_count[artist["id"]] += 1
      pbar.update(1)
    offset += 50

100%|██████████| 266/266 [00:02<00:00, 111.25it/s]


In [21]:
# sort artists_ids_count by count
sorted_artists = sorted(artists_ids_count.items(), key=lambda x: x[1], reverse=True)
for idx, (artist, count) in enumerate(sorted_artists[:10]):
  print(f"{idx + 1}. {sp.artist(artist)['name']} - {count}")

1. Nina Chuba - 12
2. NF - 10
3. ABBA - 7
4. Coldplay - 5
5. Billie Eilish - 5
6. Madison Beer - 5
7. Orelsan - 4
8. Apashe - 4
9. The Tech Thieves - 4
10. Unlike Pluto - 4


In [22]:
# count tracks by genre of the artists on the track
genres_count = defaultdict(int)
genres_artists = defaultdict(set)
for artist, count in tqdm(artists_ids_count.items()):
  artist_info = sp.artist(artist)
  for g in artist_info["genres"]:
    genres_count[g] += count
    genres_artists[g].add(artist_info["name"])

100%|██████████| 331/331 [00:27<00:00, 12.18it/s]


In [35]:
# sort genres_artists with the keys in the same order as genres_count
sorted_genres_artists = sorted(genres_artists.items(), key=lambda x: genres_count[x[0]], reverse=True)
# sort by count of artists
sorted_genres_artists_dict = {k: v for k, v in sorted(sorted_genres_artists, key=lambda x: len(x[1]), reverse=True)}
# print top 10 genres and the length of the list of artists
for idx, (genre, artists) in enumerate(list(sorted_genres_artists_dict.items())[:10]):
    print(f"{idx + 1}. {genre} - {len(artists)}")

1. pop - 54
2. dance pop - 34
3. edm - 26
4. alt z - 25
5. electropop - 23
6. electro house - 22
7. pop rap - 20
8. pop dance - 18
9. tropical house - 18
10. traprun - 16


In [41]:
import json
# convert set to list in values of dict and sort by artist name
sorted_genres_artists_dict = {k: sorted(list(v)) for k, v in sorted_genres_artists_dict.items()}
# write sorted_genres to json file with utf-8 encoding
with open("genres.json", "w", encoding="utf-8") as f:
  json.dump(sorted_genres_artists_dict, f, ensure_ascii=False, indent=2)